In [15]:
import time
from math import comb
import numpy as np
import sympy as sp
from itertools import combinations
import chipsplitting.hyperfield.utils as utils
from chipsplitting import PascalForm
from chipsplitting.hyperfield import (
    HyperfieldPascalForm as HVPascal, 
    HyperfieldHomogeneousLinearSystem as HVLinearSystem,
    HyperfieldVector
)
from chipsplitting import pairing_matrix, print_matrix, PascalForm

In [183]:
def compute_coefficients(mode, n):
    coeff = {}
    if mode == "1-t":
        for k in range(n + 1):
            coeff[f"t{k}"] = comb(n,k) * (-1)**(k)
        return coeff
    elif mode == "t":
        for k in range(n + 1):
            coeff[f"t{k}"] = 1 if k == n else 0
        return coeff
    else:
        raise ValueError()
        
def create_w_symbols(n):
    # Create a string pattern for n + 1 symbols
    pattern = ' '.join(f'w{i}' for i in range(n))
    # Create the symbols using the pattern
    symbols = sp.symbols(pattern)
    return symbols

def get_linear_equations(support):
    t = sp.symbols('t')
    ws = create_w_symbols(len(support))
    eqn = 0
    for index, coord in enumerate(support):
        col, row = coord
        expression = ws[index] * t**col * (1 - t)**row
        expanded_expression = sp.expand(expression)
        eqn += expanded_expression
    eqn += -1
    return eqn

In [ ]:
1, 1 - w4, 1, w4, w4, w5)

In [187]:
def max_degree(support):
    max_degree = 0
    for col, row in support:
        if col + row > max_degree:
            max_degree = col + row
    return max_degree


def contains_var(sol):
    for x in sol:
        if not x.is_number:
            return True
    return False
    
def valid_solution_set(solutions):
    if not solutions:
        return False

    for sol in solutions:
        if contains_var(sol):
            return False
    return True

def is_fundamental(support_as_index):
    t = sp.symbols('t')
    support = [utils.to_coordinate(s) for s in support_as_index]
    ws = create_w_symbols(len(support))
    
    max_power_t = max_degree(support)
    expression = get_linear_equations(support)
    linear_equations = {}
    
    for power in range(max_power_t + 1):
        coeff = expression.coeff(t, power)
        linear_equations[f"t{power}"] = (coeff)
    
    system = list(linear_equations.values())
    solution_set = sp.linsolve(system, *ws)

    return valid_solution_set(solution_set)

In [185]:
is_fundamental([1,3,4,5])

{(2 - w2, w2 - 1, w2, 1)}


False

**Task**: For $\Delta_2$, find all positive supports of size $3$ of valid hyperfield configurations of degree $d = 1,2,3$.

In [2]:
possible_supports = {}

def find_positive_supports(pos_support_size, mode = "fast", expand = False):
    degree_end = 2 * pos_support_size - 3
    # hyperfield variety generated by all pascal equations
    S = {}
    for d in range(1, degree_end + 1):
        num_cells = utils.gauss(d+1)
        # positive cells are all cells except the (0,0) cell
        num_pos_cells = num_cells - 1
        if num_pos_cells < pos_support_size:
            continue
            
        base_types = ["diag", "row", "col"]
        A = [HVPascal(d, b, k) for b in base_types for k in range(d + 1)]
        linear_system = HVLinearSystem(A)
        
        # I believe quick_solve_loop has a bug
        if mode == "fast":
            solutions = linear_system.quick_solve_loop(pos_support_size)
        else:
            solutions = linear_system.quick_solve(pos_support_size)
        
        if expand:
            def expand_solution(sol):
                num_selections = pos_support_size - len(sol)
                assert num_selections > 0
                domain = [x for x in range(utils.gauss(d + 1)) if x not in sol]
                combi_from_domain = list(combinations(domain, num_selections))
                return [tuple(sorted(sol + c)) for c in combi_from_domain]

            expanded_solutions = [sol for sol in solutions if len(sol) == pos_support_size]
            for sol in [sol for sol in solutions if len(sol) < pos_support_size]:
                expanded_solutions += expand_solution(sol)
            solutions = list(set(expanded_solutions))
        
        S[d] = solutions
        print(f"d={d}, #solutions={len(solutions)}")
    return S

In [3]:
n_begin = 1
n_end = 5
for pos_support_size in range(n_begin + 1, n_end + 2):
    print()
    print(f"n={pos_support_size - 1}")
    possible_supports[pos_support_size] = find_positive_supports(pos_support_size, mode = "fast", expand = True)


n=1
d=1, #solutions=1

n=2
d=2, #solutions=3
d=3, #solutions=1

n=3
d=2, #solutions=7
d=3, #solutions=20
d=4, #solutions=16
d=5, #solutions=6

n=4
d=2, #solutions=5
d=3, #solutions=70
d=4, #solutions=233
d=5, #solutions=437
d=6, #solutions=645
d=7, #solutions=784

n=5
d=3, #solutions=108
d=4, #solutions=1038
d=5, #solutions=4402
d=6, #solutions=12318
d=7, #solutions=26325
d=8, #solutions=46655
d=9, #solutions=77426


In [125]:
n = 1
for d in possible_supports[n + 1]:
    for support in possible_supports[n + 1][d]:
        if is_fundamental(support):
            print(support)

(1, 2)


In [124]:
n = 2
for d in possible_supports[n + 1]:
    for support in possible_supports[n + 1][d]:
        if is_fundamental(support):
            print(support)

(3, 4, 5)
(1, 4, 5)
(2, 3, 4)
(4, 6, 9)


In [188]:
n = 3
fundamental_models = []
start = time.time()
for d in possible_supports[n + 1]:
    for support in possible_supports[n + 1][d]:
        if is_fundamental(support):
            eqn = '0'
            for num, index in enumerate(support):
                row, col = utils.to_coordinate(index)
                eqn += f"+w{num}*t**{col}*(1-t)**{row}"
            solution = sp.solve(f"{eqn} - 1", [f"w{i}" for i in range(len(support))])
            if isinstance(solution, dict):
                A = [PascalForm(d, "col", k) for k in range(d + 1)]
                config = [solution[sp.symbols(f"w{support.index(i)}")] if i in support else 0 for i in range(utils.gauss(d + 1))]
                config[0] = -1
                solves_pascal = all([Pascal(config) == 0 for Pascal in A])
                if solves_pascal:
                    fundamental_models.append((support, solution))
end = time.time()
print(f"Elapsed time: {end - start}")
print(len(fundamental_models))

Elapsed time: 0.27698373794555664
25


In [189]:
def check_w(w):
    for x in w.values():
        if x <= 0:
            return False
    return True
len([(support, w) for support, w in fundamental_models if check_w(w)])

18

In [191]:
n = 4
fundamental_models = []
start = time.time()
for d in possible_supports[n + 1]:
    for support in possible_supports[n + 1][d]:
        if not is_fundamental(support):
            continue
        eqn = '0'
        for num, index in enumerate(support):
            row, col = utils.to_coordinate(index)
            eqn += f"+w{num}*t**{col}*(1-t)**{row}"
        solution = sp.solve(f"{eqn} - 1", [f"w{i}" for i in range(len(support))], simplify = False)
        if isinstance(solution, dict):
            # A = [PascalForm(d, "diag", k) for k in range(d + 1)]
            # config = [solution[sp.symbols(f"w{support.index(i)}")] if i in support else 0 for i in range(utils.gauss(d + 1))]
            # config[0] = -1
            # solves_pascal = all([Pascal(config) == 0 for Pascal in A])
            # if solves_pascal:
            fundamental_models.append((support, solution))
end = time.time()
print(f"Elapsed time: {end - start}")
print(len(fundamental_models))

Elapsed time: 6.578874111175537
331


In [192]:
fundamental_models_4 = fundamental_models.copy()
len(fundamental_models_4)

331

In [193]:
M = []
for supp, sol in fundamental_models_4:
    valid = True
    for x in sol.values():
        if x <= 0:
            valid = False
            break
    if valid:
        M.append((supp, sol))
len(M)

134

This looks good.

In [195]:
n = 5
fundamental_models = []
start = time.time()
for d in [3, 4, 5, 6]:
    for support in possible_supports[n + 1][d]:
        if not is_fundamental(support):
            continue
        eqn = '0'
        for num, index in enumerate(support):
            row, col = utils.to_coordinate(index)
            eqn += f"+w{num}*t**{col}*(1-t)**{row}"
        solution = sp.solve(f"{eqn} - 1", [f"w{i}" for i in range(len(support))])
        if isinstance(solution, dict):
            # A = [PascalForm(d, "diag", k) for k in range(d + 1)]
            # config = [solution[sp.symbols(f"w{support.index(i)}")] if i in support else 0 for i in range(utils.gauss(d + 1))]
            # config[0] = -1
            # solves_pascal = all([Pascal(config) == 0 for Pascal in A])
            # if solves_pascal:
            fundamental_models.append((support, solution))
end = time.time()
print(f"Elapsed time: {end - start}")
print(len(fundamental_models))

Elapsed time: 117.3024230003357
5380


In [281]:
import json

# lets store our result in a json file
# so we don't need to compute the variety again
json_file_name = 'section8_n5_d3d4d5d6.json'
with open(json_file_name, 'w') as json_file:
    json.dump([{"support": supp, "solution": {str(key) : sol[key] for key in sol}}for supp, sol in fundamental_models], json_file, default=int)

print(f"JSON dump has been written to {json_file_name}")

JSON dump has been written to section8_n5_d3d4d5d6.json


In [196]:
fundamental_models_5 = fundamental_models.copy()
M = []
for supp, sol in fundamental_models_5:
    valid = True
    for x in sol.values():
        if x <= 0:
            valid = False
            break
    if valid:
        M.append((supp, sol))
len(M)

856

Looks good, as well! Only d = 7,8,9 are missing.

## Appendix

Let's take a closer look at an example. Consider $\Delta_4$ and a support size of $5$.

```
14
10 15  
 6 11 16
 3  7 11 17
 1  4  8 12 18
 0  2  5  9 13 19
```

In [4]:
n = 4
d = 5
possible_supports[n + 1][d][-100]

(4, 5, 10, 17, 18)

We see that a possible positive support for a valid configuration may have the form

```
 0
 *  0  
 0  0  0
 0  0  0  *
 0  *  0  0  *
 0  0  *  0  0  0
```

This corresponds to a reduced statistical model $(w_\nu,i_\nu,j_\nu)_{\nu = 0}^4 \in \Delta_4$ of the form
$$
\begin{pmatrix}
(w_0, 0, 4) \\
(w_1, 1, 1) \\
(w_2, 2, 0) \\
(w_3, 3, 2) \\
(w_4, 4, 1)
\end{pmatrix} \in \Delta_4.
$$
So, this model corresponds to probability mass function
$$
\begin{align*}
p_0 &= w_0(1 - t)^{4} \\
p_1 &= w_1 t (1 - t) \\
p_2 &= w_2 t^2 \\
p_3 &= w_3 t^3 (1 - t)^2 \\
p_4 &= w_4 t^4 (1 - t)
\end{align*}.
$$
We want to solve this system of polynomial equations under the contraint $p_0 + p_1 + ... + p_4 = 1$.

In [50]:
import sympy as sp

# Define the variables
t = sp.symbols('t')
w0, w1, w2, w3, w4 = sp.symbols('w0 w1 w2 w3 w4')

# Define the polynomials
p0 = w0 * (1 - t)**4
p1 = w1 * t * (1 - t)
p2 = w2 * t**2
p3 = w3 * t**3 * (1 - t)**2
p4 = w4 * t**4 * (1 - t)

# Define the constraint equation
constraint = p0 + p1 + p2 + p3 + p4 - 1

# Solve the constraint equation for t
solution = sp.solve(constraint, [w0, w1, w2, w3, w4])

# Display the solution
solution

[((t**5*w4 - t**4*w4 - t**3*w3*(t - 1)**2 + t**2*w1 - t**2*w2 - t*w1 + 1)/(t - 1)**4,
  w1,
  w2,
  w3,
  w4)]

We see that a possible positive support for a valid configuration may have the form

```
 *  
 0  0  
 0  *  0  
 0  0  0  *  
```

This corresponds to a reduced statistical model $(w_\nu,i_\nu,j_\nu)_{\nu = 0}^4 \in \Delta_4$ of the form
$$
\begin{pmatrix}
(w_0, 1, 1) \\
(w_1, 3, 0) \\
(w_2, 0, 3)
\end{pmatrix} \in \Delta_4.
$$
So, this model corresponds to probability mass function
$$
\begin{align*}
p_0 &= w_0t(1 - t) \\
p_1 &= w_1 t^3 \\
p_2 &= w_2 (1-t)^3
\end{align*}.
$$
We want to solve this system of polynomial equations under the contraint $p_0 + p_1 + p_2 = 1$.

In [55]:
# Define the variables
t = sp.symbols('t')
w0, w1, w2 = sp.symbols('w0 w1 w2')

# Define the polynomials
p0 = w0 * t ** 2
p1 = w1 * t * (1 - t)
p2 = w2 * (1-t)**2

# Define the constraint equation
constraint = p0 + p1 + p2 - 1

# Solve the constraint equation for w
solution = sp.solve(constraint, [w0, w1, w2])

# Display the solution
solution

{w0: 1, w1: 2, w2: 1}